# Access S2S API

This notebook walks through an example of sending an API call directly to S2S, instead of through the client. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/worldbank/DECAT_Space2Stats/blob/main/docs/user-docs/space2stats_api_demo.ipynb)

In [1]:
# !pip install folium mapclassify matplotlib

In [1]:
from typing import Dict

import folium as flm  # Comment out if you have not installed, or run pip install folium matplotlib mapclassify
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from geojson_pydantic import Feature, Polygon
from shapely import from_geojson

ImportError: cannot import name 'field_validator' from 'pydantic' (c:\WBG\Anaconda3\envs\s2s_ingest\lib\site-packages\pydantic\__init__.cp310-win_amd64.pyd)

In [3]:
BASE_URL = "https://space2stats.ds.io"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [4]:
response = requests.get(FIELDS_ENDPOINT)
if response.status_code != 200:
    raise Exception(f"Failed to get fields: {response.text}")

available_fields = response.json()
print("Available Fields:", available_fields)

Available Fields: ['sum_built_area_m_2030', 'ogc_fid', 'sum_pop_f_0_2020', 'sum_pop_f_10_2020', 'sum_pop_f_15_2020', 'sum_pop_f_1_2020', 'sum_pop_f_20_2020', 'sum_pop_f_25_2020', 'sum_pop_f_30_2020', 'sum_pop_f_35_2020', 'sum_pop_f_40_2020', 'sum_pop_f_45_2020', 'sum_pop_f_50_2020', 'sum_pop_f_55_2020', 'sum_pop_f_5_2020', 'sum_pop_f_60_2020', 'sum_pop_f_65_2020', 'sum_pop_f_70_2020', 'sum_pop_f_75_2020', 'sum_pop_f_80_2020', 'sum_pop_m_0_2020', 'sum_pop_m_10_2020', 'sum_pop_m_15_2020', 'sum_pop_m_1_2020', 'sum_pop_m_20_2020', 'sum_pop_m_25_2020', 'sum_pop_m_30_2020', 'sum_pop_m_35_2020', 'sum_pop_m_40_2020', 'sum_pop_m_45_2020', 'sum_pop_m_50_2020', 'sum_pop_m_55_2020', 'sum_pop_m_5_2020', 'sum_pop_m_60_2020', 'sum_pop_m_65_2020', 'sum_pop_m_70_2020', 'sum_pop_m_75_2020', 'sum_pop_m_80_2020', 'sum_pop_m_2020', 'sum_pop_f_2020', 'sum_pop_2020', 'pop', 'pop_flood', 'pop_flood_pct', 'ghs_11_count', 'ghs_12_count', 'ghs_13_count', 'ghs_21_count', 'ghs_22_count', 'ghs_23_count', 'ghs_30_co

In [ ]:
AOIModel = Feature[Polygon, Dict]
# https://geojson.io/ - optional site for extracting AOIs in geojson format
# ~burundi
minx, miny, maxx, maxy = 29.038924, -4.468958, 30.850461, -2.310523

aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [minx, maxy],
                [minx, miny],
                [maxx, miny],
                [maxx, maxy],
                [minx, maxy],
            ]
        ],
    },
    "properties": {"name": "Updated AOI"},
}


feat = AOIModel(**aoi)

In [6]:
# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "touches",
    "fields": ["sum_pop_2020"],
    "geometry": "polygon",
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
df = pd.DataFrame(summary_data)

In [7]:
df["geometry"] = df["geometry"].apply(lambda geom: from_geojson(geom))
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
gdf.head()

,hex_id,geometry,sum_pop_2020
0,866ad8087ffffff,"POLYGON ((30.10399 -2.3257, 30.11745 -2.29278,...",25345.874701
1,866ad808fffffff,"POLYGON ((30.16219 -2.34876, 30.17563 -2.31584...",11870.360712
2,866ad8097ffffff,"POLYGON ((30.05471 -2.36355, 30.06817 -2.33062...",19034.332476
3,866ad809fffffff,"POLYGON ((30.11291 -2.38662, 30.12636 -2.35369...",14700.551092
4,866ad80c7ffffff,"POLYGON ((30.26964 -2.33398, 30.28307 -2.30106...",12067.935215


In [8]:
m = gdf.explore(
    column="sum_pop_2020",
    tooltip="sum_pop_2020",
    cmap="YlGnBu",
    legend=True,
    scheme="naturalbreaks",
    legend_kwds=dict(colorbar=True, caption="Population", interval=False),
    style_kwds=dict(weight=0, fillOpacity=0.8),
    name="Population by Hexagon",
)
flm.LayerControl("topright", collapsed=False).add_to(m)
m